# Python Timelapse Project 

In [ ]:
#importing required libraries
import time
import datetime
from datetime import date 
import requests
import urllib.request
from bs4 import BeautifulSoup
from PIL import Image,ImageDraw
import glob
import os

In [ ]:
#function for date validation

def date_validation(day,month,year):
    isValidDate = True
    try:
        datetime.datetime(int(year),int(month),int(day)) 
    except ValueError:
        isValidDate = False
    if(isValidDate):
        return True
    else:
        return False 

In [ ]:
#fetching present date

today = date.today() 
today=str(today)
year,month,day = today.split('-')

In [ ]:
#Function to create gif
def create_gif(i_date):
    speed = 100
    x = int(input("Choose gif speed -\n1. Slow\n2. Normal\n3. Fast "))
    if x==1:
        speed = 200
    elif x==2:
        speed = 100
    elif x==3:
        speed = 50
    else:
        print("Wrong input (Normal is the default speed)")
    frame = []
    imgs = glob.glob("D:\\{}\\*.jpg".format(i_date))
    print("Creating gif with specified speed...")
    for i in imgs:
        new_frame = Image.open(i)
        frame.append(new_frame)
        frame[0].save('D:\\{}\\{}.gif'.format(i_date,i_date),format="GIF",append_images=frame[1:],save_all=True,duration=speed,loop=0)
    print("Timelapse video successfully created.\nPlease check in " + i_date + " folder in D drive.") 
create_gif(20200606)

In [ ]:
#Using requests and beautifulsoup, retrieving images

url2 = 'https://sdo.gsfc.nasa.gov/assets/img/browse/'
headers = {
    'Accept-Encoding': 'gzip, deflate, sdch',
    'Accept-Language': 'en-US,en;q=0.8',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.1 Safari/605.1.15',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Referer': 'http://www.google.com/',
    'Connection': 'keep-alive',
}

def fetch_img(i_day,i_month,i_year,i_date):
    url = 'https://sdo.gsfc.nasa.gov/assets/img/browse/index.php?b={}%2F{}%2F{}'.format(i_year,i_month,i_day)
    r = requests.get(url=url, headers=headers)

    soup = BeautifulSoup(r.text, 'html.parser')
    
    i = 0

    for img in soup.findAll('a'):
        i = i+1
        image_temp = img.get('href')
        i_url,e_url = image_temp.split(".")
        #Checking the required telescope and image resolution
        if i_url[-1:-9:-1] == '1710_215': 
            image_path = url2 + i_url + '.jpg'
            local_path = "D:\\{}\\{}.jpg".format(i_date,i)
            with open(local_path,'wb') as f:
                f.write(requests.get(url = image_path).content)
            f.close()
        else:
            pass
    print("Images dowloaded. Now CREATING GIF..")
    #calling function to create gif
    create_gif(i_date)

In [ ]:
print('''
  _______ _____ __  __ ______   _               _____   _____ ______ 
 |__   __|_   _|  \/  |  ____| | |        /\   |  __ \ / ____|  ____|
    | |    | | | \  / | |__    | |       /  \  | |__) | (___ | |__   
    | |    | | | |\/| |  __|   | |      / /\ \ |  ___/ \___ \|  __|  
    | |   _| |_| |  | | |____  | |____ / ____ \| |     ____) | |____ 
    |_|  |_____|_|  |_|______| |______/_/    \_\_|    |_____/|______| ''')
                                                                     
date = input('Input a date between 2011-2021 (YYYY-MM-DD): ')
#slicing input dates
i_year, i_month, i_day = date.split('-')
i_date = i_year+i_month+i_day

#if input year == current year and month > current month -> Error
#if input year == current year and month = current month and input date > current date -> Error
#if input year == current year and month = current month and input date < current date -> Fetch images
#if input year == current yeear and month < current month -> Fetch images for any valid date
#i_year = input year
#year = current year

if(i_year == year):
    if(i_month <= month):
        if(i_day <day):
            print("FETCHING IMAGES....")
            os.mkdir('D:\{}'.format(i_date))
            fetch_img(i_day,i_month,i_year,i_date) 
        else:
            print("\n ERROR: Invalid Date")
    else:
        print("\n ERROR: Invalid Date")
elif(i_year < year and i_year > '2011' and date_validation(i_day,i_month,i_year)):
    print("FETCHING IMAGES....")
    os.mkdir('D:\{}'.format(i_date))
    fetch_img(i_day,i_month,i_year,i_date)
else:
    print("Please input a correct date")